In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="6"

In [2]:
from sklearn import svm
import torch
from torch import optim
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn.init as init
import random
import pickle
import warnings 
warnings.filterwarnings(action='ignore')
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
import math
from torch.utils.data.sampler import Sampler
import itertools
import tqdm
from torch.utils.data import TensorDataset
#device = torch.device('cuda:7' if torch.cuda.is_available() else 'cpu')
#print(device)

In [3]:
# Clinical information embedding
def clinic_concat(data,info):
    for i in range(0,int(data.shape[1]/shell)+1):
        index1 = shell*i
        index2 = index1 + (shell -1)
        if index2 > data.shape[1]:
            index2 = data.shape[1]
        tmp = data.loc[:,index1:index2]
        tmp = pd.concat([info,tmp],axis=1)
        if i==0:
            data_concat = tmp
        else:
            data_concat = pd.concat([data_concat,tmp],axis=1)
    return data_concat

In [4]:
def create_triplet(dat1,dat2,n):
    triplet = []
    # Anchor = group1 (Disease)
    for x in range(int(n/2)):
        anchor_index = x%dat1.shape[0]               
        while True:
            pos_index = random.randint(0,dat1.shape[0]-1)
            if anchor_index != pos_index:
                break
        neg_index = random.randint(0,dat2.shape[0]-1)
        triplet.append([ dat1[anchor_index,],dat1[pos_index,],dat2[neg_index,] ])
    
    # Anchor = group2 (Control)
    for x in range(int(n/2)):
        anchor_index = x%dat2.shape[0]               
        while True:
            pos_index = random.randint(0,dat2.shape[0]-1)
            if anchor_index != pos_index:
                break
        neg_index = random.randint(0,dat1.shape[0]-1)
        triplet.append([ dat2[anchor_index,],dat2[pos_index,],dat1[neg_index,] ])
    
    return triplet

In [5]:
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.best_model = None

    def __call__(self, val_loss, model):

        score = val_loss

        if self.best_score is None:
            self.best_score = score
            self.best_model = model
            self.val_loss_min = val_loss
        elif score > self.best_score + self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.val_loss_min = val_loss
            self.best_model = model
            

In [6]:
class CNN_1L(nn.Module):
    def __init__(self,feature_dimension=100):
        super(CNN_1L, self).__init__()

        self.conv = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size= (shell+2), stride= (shell+2), padding=0)
            #nn.BatchNorm1d(16)
            
            )
        self.pool = nn.MaxPool1d(2,stride=2)
        self.fc = nn.Linear(656, feature_dimension)
        self.act = nn.Sigmoid()
            
    def forward(self, x):
        # expected conv1d input : minibatch_size x num_channel x width
        
        x = x.view(x.shape[0], 1,-1)
        out = self.conv(x)
        out = self.act(out)
        out = self.pool(out)
        out = out.view(x.shape[0], out.size(1) * out.size(2))
        logit = self.fc(out)
        
        return logit
   

In [7]:
class Triple_Distance(nn.Module):
    def __init__(self, cosine = False):
        super().__init__()
        self.cosine = cosine
        
    def forward(self, anchor, positive, negative):
        # Use Cosine Distance
        if self.cosine:
            self.margin = 1
            # L2 Normalization
            anchor_normed = F.normalize(anchor, p=2, dim=1)
            positive_normed = F.normalize(positive, p=2, dim=1)
            negative_normed = F.normalize(negative, p=2, dim=1)
            
            # Cosine Distance
            distance_positive = torch.sum(torch.mul(anchor_normed, positive_normed), dim=1, keepdim=True)
            distance_negative = torch.sum(torch.mul(anchor_normed, negative_normed), dim=1, keepdim=True)
            
        # Use Euclidian Distance
        else:
            distance_positive = (anchor - positive).pow(2).sum(1)  
            distance_negative = (anchor - negative).pow(2).sum(1)  

        return distance_positive, distance_negative


In [8]:
class Mysampler(Sampler):
    def __init__(self, dataset,index1):
        self.index1 = index1
        
    def __iter__(self):
        random.shuffle(self.index1)
        
        
        
        
        return iter(self.index1)
    
    def __len__(self):
        return (len(self.index1) )

In [9]:
def getIntersection(a,b):
    indices = torch.zeros_like(a, dtype=torch.uint8)
    
    for elem in b:
        indices = indices| (a==elem).type(torch.uint8)

    intersection = a[indices.type(torch.bool)]
    return intersection

In [10]:
def train_model(model,patience,n_epochs, deg):
    train_losses = []
    valid_losses = []
    avg_train_losses = []
    avg_valid_losses = []
    degree = deg

    train_len = []
    
    distance = Triple_Distance()
    early_stopping = EarlyStopping(patience=patience, verbose=True)
    model.train()
    
    for epoch in range(1, n_epochs + 1):
        optimizer.zero_grad()
        
        ###################
        # train the model #
        ###################
        total_pos = []
        total_neg = []
        for i,data in enumerate(train_dataloader):
            
            anchor, pos, neg = data
            anchor, pos, neg = anchor.float().cuda(), pos.float().cuda(), neg.float().cuda()

            
            #########################################
            # Loss pre-calculate for triplet mining #
            #########################################
            
            with torch.no_grad():
                # Forward pass
                output1 = model(anchor)
                output2 = model(pos)
                output3 = model(neg)

            
            
                distance_pos, distance_neg = distance(output1,output2,output3)
                distance_pos_s, distance_neg_s = distance(output2,output1,output3)

                target = torch.FloatTensor(distance_pos.size()).fill_(-1).cuda()
                target = Variable(target)
                
                distance_pos = distance_pos.squeeze()
                distance_neg = distance_neg.squeeze()
                distance_pos_s = distance_pos_s.squeeze()
                distance_neg_s = distance_neg_s.squeeze()

                if i==0:
                    total_pos = distance_pos
                    total_neg = distance_neg

                    total_pos_s = distance_pos_s
                    total_neg_s = distance_neg_s

                else:
                    total_pos = torch.cat([total_pos,distance_pos])
                    total_neg = torch.cat([total_neg,distance_neg])
        
                    total_pos_s = torch.cat([total_pos_s, distance_pos_s])
                    total_neg_s = torch.cat([total_neg_s, distance_neg_s])
        index1_1 = (total_pos < total_neg).nonzero().flatten()
        index1_2 = (total_neg < (total_pos + margin)).nonzero().flatten()
        
        index1 = getIntersection(index1_1, index1_2)

        index1_1 = (total_pos_s < total_neg_s).nonzero().flatten()
        index1_2 = (total_neg_s < (total_neg_s + margin)).nonzero().flatten()

        index1_s = getIntersection(index1_1,index1_2)
        index2 = ((total_pos+margin) < total_neg).nonzero().flatten()
        index1 = getIntersection(index1,index1_s)
        
        
        if len(index1) < 10:
            break

        # Sampler define
        my_sampler = Mysampler(train_trip,index1)
        my_loader = torch.utils.data.DataLoader(train_trip, sampler = my_sampler, batch_size = train_batch_size)
        
        for i,data in enumerate(my_loader):
            anchor, pos, neg = data
            anchor, pos, neg = anchor.float().cuda(), pos.float().cuda(), neg.float().cuda()
            
            # Forward pass
            output1 = model(anchor)
            output2 = model(pos)
            output3 = model(neg)
        
            output_cen = torch.true_divide((output1+output2),2)
            distance_pos, distance_neg = distance(output1, output2, output3)
            distance_pos_s, distance_neg_s = distance(output2,output1,output3)
            
            distance_cen_pos,distance_cen_neg = distance(output_cen,output2,output3)
            tana = math.tan(math.pi *(degree/180))
            tmp = distance_pos -4*(tana**2)*distance_cen_neg
            loss_ang = tmp[tmp>0].sum()
            
            target = torch.FloatTensor(distance_pos.size()).fill_(-1)
            target = target.cuda()
            target = Variable(target)
            loss = criterion(distance_pos, distance_neg, target) + criterion(distance_pos_s,distance_neg_s,target) + loss_ang
            

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())
        
        ######################    
        # validate the model #
        ######################
        with torch.no_grad():
            model.eval() # prep model for evaluation
            total_pos = []
            total_neg = []
            for i,data in enumerate(valid_dataloader):
                anchor, pos, neg = data
                anchor, pos, neg = anchor.float(), pos.float(), neg.float()
                anchor, pos, neg = anchor.cuda(), pos.cuda(), neg.cuda()
                
                
                ###################
                # Loss pre-calculate
                ###################
                               
                # Forward pass
                output1 = model(anchor)
                output2 = model(pos)
                output3 = model(neg)
                
                distance_pos, distance_neg = distance(output1, output2, output3)
                distance_pos_s, distance_neg_s = distance(output2,output1,output3)
                
                target = torch.FloatTensor(distance_pos.size()).fill_(-1)
                target = target.cuda()
                target = Variable(target)
                
                distance_pos = distance_pos.squeeze()
                distance_neg = distance_neg.squeeze()
                distance_pos_s = distance_pos_s.squeeze()
                distance_neg_s = distance_neg_s.squeeze()


                if i==0:
                    total_pos = distance_pos
                    total_neg = distance_neg

                    total_pos_s = distance_pos_s
                    total_neg_s = distance_neg_s
                else:
                    total_pos = torch.cat([total_pos,distance_pos])
                    total_neg = torch.cat([total_neg,distance_neg])
                    
                    total_pos_s = torch.cat([total_pos_s,distance_pos_s])
                    total_neg_s = torch.cat([total_neg_s, distance_neg_s])
            
            # Semi hard
            index1_1 = (total_pos < total_neg).nonzero().flatten()
            index1_2 = (total_neg < (total_pos + margin)).nonzero().flatten()
        
            index1 = getIntersection(index1_1, index1_2)

            index1_1 = (total_pos_s <total_neg_s).nonzero().flatten()
            index1_2 = (total_neg_s <(total_pos_s + margin)).nonzero().flatten()
            
            index1_s = getIntersection(index1_1,index1_2)
            index1 = getIntersection(index1,index1_s)

            my_sampler2 = Mysampler(valid_trip,index1)
            
            my_loader2 = torch.utils.data.DataLoader(valid_trip, sampler = my_sampler2, batch_size = valid_batch_size)
            
            
            for i,data in enumerate(my_loader2):    
                anchor, pos, neg = data
                anchor, pos, neg = anchor.float(), pos.float(), neg.float()
                anchor, pos, neg = anchor.cuda(), pos.cuda(), neg.cuda()
                
                # Forward pass
                output1 = model(anchor)
                output2 = model(pos)
                output3 = model(neg)
                
                output_cen = torch.true_divide((output1+output2),2)

                distance_pos, distance_neg = distance(output1, output2, output3)
                distance_pos_s, distance_neg_s = distance(output2,output1,output3) 
                distance_cen_pos,distance_cen_neg = distance(output_cen,output2,output3)

                target = torch.FloatTensor(distance_pos.size()).fill_(-1)
                target = target.cuda()
                target = Variable(target)
                
                tana = math.tan(math.pi*(degree/180))
                tmp = distance_pos-4*(tana**2)*distance_cen_neg
                loss_ang = tmp[tmp>0].sum()

                loss = criterion(distance_pos, distance_neg, target) + criterion(distance_pos_s, distance_neg_s,target) + loss_ang
                valid_losses.append(loss.item())
                
                
        train_loss = np.average(train_losses)
        valid_loss = np.average(valid_losses)
        avg_train_losses.append(train_loss)
        avg_valid_losses.append(valid_loss)
        
        epoch_len = len(str(n_epochs))
        
        print_msg = (f'[{epoch:>{epoch_len}}/{n_epochs:>{epoch_len}}] ' +
                     f'train_loss: {train_loss:.5f} ' +
                     f'valid_loss: {valid_loss:.5f}')
        # print(print_msg)
       
        # clear lists to track next epoch
        train_losses = []
        valid_losses = []
        
        early_stopping(valid_loss, model)
        if early_stopping.early_stop:
            break
    
    # load the last checkpoint with the best model
    best_model = early_stopping.best_model
    
    return  best_model, early_stopping.best_score


In [11]:
def weights_init(m):
    if isinstance(m, nn.Conv1d):
        torch.nn.init.xavier_uniform_(m.weight)
        torch.nn.init.zeros_(m.bias)
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        torch.nn.init.zeros_(m.bias)

In [12]:
shell = 200
num_train_samples = 30000
num_valid_samples = 10000

In [13]:
best_perform = 0
margin = 0.5
train_batch_size = 1000
drop_prob = 0.0
valid_batch_size = 1000
rg = 0.0001
degree_list = [45,60]
feature_dimension_list = [60,90,150,300]
learning_rate_list = [0.01,0.005]

patience = 100
n_epochs = 5000

In [14]:
class Net(nn.Module):
    def __init__(self,feature_dimension):
        super(Net, self).__init__()
        
        #self.act = nn.RReLU()
        self.fc = nn.Linear(feature_dimension, 1) 
        #self.dropout = torch.nn.Dropout(p=drop_prob)
               
        
    
    def forward(self, x):
        
        out = self.fc(x)
        #out = self.dropout(out)
        #out = self.act(out)
        out = out.view(-1,)
        
        
        return out


In [15]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
x = pd.read_csv("./dat.csv",header=None)
y = pd.read_csv("./label.txt",header=None)
info = pd.read_csv("./dat_info.csv",header=None)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
x_concat = clinic_concat(x,info)

In [16]:
tr_index = []
te_index = []
for index1,index2 in skf.split(x_concat,y):
    tr_index.append(index1)
    te_index.append(index2)

In [17]:
auc = []
param_list = []
for fold_ in range(1,6):
    iter_auc = []
    print("Fold: ", str(fold_))
    
    train_index = tr_index[fold_-1]
    test_index = te_index[fold_-1]
    valid_index = random.sample(list(train_index), int(0.2*len(train_index)))
    train_index = np.setdiff1d(train_index,valid_index)
    
    train_x_concat = x_concat.loc[train_index,]
    train_y = y.loc[train_index]
    group_index = (train_y.squeeze()==0)
    train_x1 = train_x_concat.loc[group_index,:].to_numpy()
    train_x2 = train_x_concat.loc[-group_index,:].to_numpy()
    
    valid_x_concat = x_concat.loc[valid_index,]
    valid_y = y.loc[valid_index]
    group_index = (valid_y.squeeze()==0)
    valid_x1 = valid_x_concat.loc[group_index,:].to_numpy()
    valid_x2 = valid_x_concat.loc[-group_index,:].to_numpy()

    test_x_concat = x_concat.loc[test_index,]
    test_y = y.loc[test_index]
    

    # Create Triplets
    train_trip = create_triplet(train_x1,train_x2,num_train_samples)
    valid_trip = create_triplet(valid_x1,valid_x2,num_valid_samples)


    train_dataloader = torch.utils.data.DataLoader(train_trip, batch_size = 300,shuffle=False)
    valid_dataloader = torch.utils.data.DataLoader(valid_trip, batch_size= 50, shuffle=False)

    train_y = torch.FloatTensor(train_y.values.reshape(-1)).cuda()
    valid_y = torch.FloatTensor(valid_y.values.reshape(-1)).cuda()
    test_y = torch.FloatTensor(test_y.values.reshape(-1)).cuda()
        
        
    # Search best parameter from validation set
    val_set = list(itertools.product(degree_list,feature_dimension_list,learning_rate_list))
    best_valid = 9999
    
    for iter_ in tqdm.tqdm(range(0,len(val_set))):
        # print(str(x)+"th Searching")
        val_par = val_set[iter_]
        
        net_ = CNN_1L(val_par[1]).cuda()
        net_.apply(weights_init)
        criterion = torch.nn.MarginRankingLoss(margin=margin)
        optimizer = optim.Adam(net_.parameters(),lr = val_par[2])
        _, best_score = train_model(net_, patience, n_epochs, val_par[0])
            
        if best_score < best_valid:
            best_valid = best_score
            save_degree = val_par[0]
            save_feature_dimension = val_par[1]
            save_learning_rate = val_par[2]
    param_list.append([save_degree,save_feature_dimension,save_learning_rate])        
    print("Search finish")
    
    for iter_ in range(0,3):
        # Train Embedding function
        net = CNN_1L(save_feature_dimension).cuda()
        net.apply(weights_init)
        criterion = torch.nn.MarginRankingLoss(margin= margin)

        optimizer = optim.Adam(net.parameters(),lr = save_learning_rate)


        net, _ = train_model(net,patience,n_epochs, save_degree)

        train_x = net(torch.tensor(train_x_concat.values).float().cuda()).detach()
        valid_x = net(torch.tensor(valid_x_concat.values).float().cuda()).detach()
        test_x = net(torch.tensor(test_x_concat.values).float().cuda()).detach()

        clf = Net(save_feature_dimension).cuda()
        clf.apply(weights_init)
        optimizer = torch.optim.Adam(clf.parameters(), lr = 0.005)
        loss_func = nn.BCEWithLogitsLoss(weight = None, size_average=None,reduce=None,reduction='mean')

        emb_train_loader = torch.utils.data.DataLoader(TensorDataset(train_x,train_y), batch_size=50)
        emb_valid_loader = torch.utils.data.DataLoader(TensorDataset(valid_x,valid_y), batch_size=30)

        train_losses = []
        valid_losses = []
        early_stopping = EarlyStopping(patience = patience, verbose = False)
        clf.train()

        for epoch in range(1,n_epochs+1):
            optimizer.zero_grad()
            
            for i,data in enumerate(emb_train_loader):
                x_ , y_ = data
                output = clf(x_)

                loss = loss_func(output,y_)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                train_losses.append(loss.item())

            with torch.no_grad():
                clf.eval()
                for i,data in enumerate(emb_valid_loader):
                    x_ , y_ = data
                    output = clf(x_)
                    loss = loss_func(output,y_)

                    valid_losses.append(loss.item())

            train_loss = np.average(train_losses)
            valid_loss = np.average(valid_losses)

            epoch_len = len(str(n_epochs))

            print_msg = (f'[{epoch:>{epoch_len}}/{n_epochs:>{epoch_len}}] ' +
                                    f'train_loss: {train_loss:.5f} ' +
                                    f'valid_loss: {valid_loss:.5f}')

            #print(print_msg)
            train_losses = []
            valid_losses = []
            early_stopping(valid_loss, clf)

            if early_stopping.early_stop:
                break

        # load the last checkpoint with the best model
        clf = early_stopping.best_model
        pred = torch.sigmoid(clf(test_x))
        pred = pred.data
        pred = pred.cpu().numpy()
        roc_auc = roc_auc_score(test_y.cpu().numpy(),pred)
        iter_auc.append(roc_auc)
        #print(roc_auc)

    print(str(fold_)+f"th Fold AUC Triplet embbedding(JTSC):{sum(iter_auc)/len(iter_auc):.3f}")
    auc.append( sum(iter_auc)/len(iter_auc) )
    

print("---------------------------------------------------------")
print("Average Triplet embedding(JTSC) AUC: ", sum(auc)/len(auc))

Fold:  1


100%|██████████| 16/16 [8:39:58<00:00, 1949.92s/it]  


Search finish
1th Fold AUC Triplet embbedding(JTSC):0.886
Fold:  2


100%|██████████| 16/16 [8:46:52<00:00, 1975.75s/it]  


Search finish
2th Fold AUC Triplet embbedding(JTSC):0.745
Fold:  3


...

3th Fold AUC Triplet embbedding(JTSC):0.759
Fold:  4


100%|██████████| 16/16 [6:59:18<00:00, 1572.39s/it]  


Search finish
4th Fold AUC Triplet embbedding(JTSC):0.850
Fold:  5


100%|██████████| 16/16 [6:19:59<00:00, 1424.95s/it]  


Search finish
5th Fold AUC Triplet embbedding(JTSC):0.811
---------------------------------------------------------
Average Triplet embedding(JTSC) AUC:  0.8102632844738107


In [18]:
for i in range(1,(len(param_list)+1)):
    print("Fold"+str(i)+": ","[degree",param_list[i-1][0],", embedding dimension",param_list[i-1][1],",learning_rate",param_list[i-1][2],"]")

Fold1:  [degree 45 , embedding dimension 150 ,learning_rate 0.01 ]
Fold2:  [degree 45 , embedding dimension 90 ,learning_rate 0.005 ]
Fold3:  [degree 60 , embedding dimension 300 ,learning_rate 0.005 ]
Fold4:  [degree 60 , embedding dimension 150 ,learning_rate 0.01 ]
Fold5:  [degree 45 , embedding dimension 300 ,learning_rate 0.01 ]


In [19]:
def cross_svm(train_x,train_y,valid_x,valid_y,test_x,test_y):
    p1 = ["linear","poly","rbf","sigmoid"] #kernel
    p2 = ['auto','scale'] #gamma
    p3 = [0.001,0.01,0.005] #tol
    best = 0.0
    val_set = list(itertools.product(p1,p2,p3))
    for i in range(0,len(val_set)):
        val_par = val_set[i]
        clf = svm.SVC(kernel=val_par[0], gamma=val_par[1], tol=val_par[2])
        
        clf.fit(train_x,train_y)
        y_pred = clf.predict(valid_x)
        
        roc_auc = roc_auc_score(valid_y,y_pred)
        if roc_auc > best:
            best = roc_auc
            y_pred_ = clf.predict(test_x)
            res = roc_auc_score(test_y,y_pred_)
                    
    return res

In [20]:
res = []
fold = 5
for fold_ in range(1,fold+1):
    
    train_index = tr_index[fold_-1]
    test_index = te_index[fold_-1]
    valid_index = random.sample(list(train_index), int(0.2*len(train_index)))
    train_index = np.setdiff1d(train_index,valid_index)
    
    train_x_concat = pd.concat([x,info],axis=1).loc[train_index,]
    train_y = y.loc[train_index]
    
    valid_x_concat = pd.concat([x,info],axis=1).loc[valid_index,]
    valid_y = y.loc[valid_index]
    
    test_x_concat = pd.concat([x,info],axis=1).loc[test_index,]
    test_y = y.loc[test_index]
        
    x_train = train_x_concat.to_numpy()
    x_valid = valid_x_concat.to_numpy()
    x_test = test_x_concat.to_numpy()
        
    y_train = train_y.values.reshape(-1)
    y_valid = valid_y.values.reshape(-1)
    y_test = test_y.values.reshape(-1)
        
               
        
    auc = cross_svm(x_train,y_train,x_valid,y_valid,x_test,y_test)
    res.append(auc)
    print("Fold:",str(fold_),"test AUC:",auc)

print("SVM Average AUC : " ,sum(res)/len(res))

Fold: 1 test AUC: 0.6586956521739131
Fold: 2 test AUC: 0.6341614906832299
Fold: 3 test AUC: 0.7524844720496894
Fold: 4 test AUC: 0.6770963704630788
Fold: 5 test AUC: 0.6240409207161125
SVM Average AUC :  0.6692957812172048
